<a href="https://colab.research.google.com/github/ztokar/Pytorch-tutorial/blob/main/CausalImpact.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
!pip install tfcausalimpact

In [2]:
import yfinance as yf
from causalimpact import CausalImpact
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [3]:
#dates
treatment_start="2020-10-20"
treatment_end="2020-10-23"
training_start="2020-09-01"
training_end="2020-10-19"
end_stock="2020-10-24"

In [13]:
#ticker info
y=["BTC-USD"]
y=yf.download(tickers=y, start=training_start,end=end_stock,interval='1d')
y=y['Adj Close'].rename('y')
y[:5]

[*********************100%***********************]  1 of 1 completed


Date
2020-09-01    11970.478516
2020-09-02    11414.034180
2020-09-03    10245.296875
2020-09-04    10511.813477
2020-09-05    10169.567383
Name: y, dtype: float64

In [21]:
stocks=['ZAL.DE','SQ','CRSP','TRMB','JD','DE','KTOS','GOOG']
X=yf.download(tickers=stocks, start=training_start,end=end_stock,interval='1d')
X=X.iloc[:,:len(stocks)]
X.head(1)

[*********************100%***********************]  8 of 8 completed


Adj Close                       ...                          
                 CRSP          DE     GOOG  ...          SQ   TRMB ZAL.DE
Date                                        ...                          
2020-09-01  93.419998  210.029678  83.0355  ...  166.660004  53.41   77.0

[1 rows x 8 columns]

In [22]:
X.columns=X.columns.droplevel()
X.head()

,CRSP,DE,GOOG,JD,KTOS,SQ,TRMB,ZAL.DE
Date,,,,,,,,
2020-09-01,93.419998,210.029678,83.035500,79.267685,19.700001,166.660004,53.410000,77.000000
2020-09-02,93.930000,209.470093,86.414001,79.959557,19.910000,162.880005,54.310001,77.000000
2020-09-03,85.690002,203.391754,82.092003,75.952454,19.520000,152.860001,50.900002,75.080002
2020-09-04,82.019997,203.903152,79.552002,76.874954,19.260000,146.389999,49.959999,71.739998
2020-09-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.000000


In [23]:
X.index=X.index.tz_localize(None)

In [25]:
df=pd.concat([y,X],axis=1).dropna()
df.head()

,y,CRSP,DE,GOOG,JD,KTOS,SQ,TRMB,ZAL.DE
Date,,,,,,,,,
2020-09-01,11970.478516,93.419998,210.029678,83.035500,79.267685,19.700001,166.660004,53.410000,77.000000
2020-09-02,11414.034180,93.930000,209.470093,86.414001,79.959557,19.910000,162.880005,54.310001,77.000000
2020-09-03,10245.296875,85.690002,203.391754,82.092003,75.952454,19.520000,152.860001,50.900002,75.080002
2020-09-04,10511.813477,82.019997,203.903152,79.552002,76.874954,19.260000,146.389999,49.959999,71.739998
2020-09-08,10131.516602,81.459999,202.841827,76.619499,73.242615,19.230000,139.110001,49.139999,73.440002
